In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from pprint import pprint
from voc_eval import parse_rec
from train_img_generator import TrainImageGenerator
from feature_generator import FeatureGenerator

In [4]:
with open('datasets/ImageSets/train.txt', 'r') as train_text:
    train_image_indices = train_text.read().strip().split('\n')

training_annotation_path = 'datasets/Annotations/{}.xml'
training_annotations_dict = {
    'waldo': {},
    'wenda': {},
    'wizard': {}
}

for train_image_index in train_image_indices:
    annotation_path = training_annotation_path.format(train_image_index)
    for label in parse_rec(annotation_path):
        target = label['name']
        bbox = label['bbox']
        if train_image_index not in training_annotations_dict[target]:
            training_annotations_dict[target][train_image_index] = [bbox]
        else:
            training_annotations_dict[target][train_image_index].append(bbox)

In [6]:
for target in training_annotations_dict.keys():
    training_image_generator = TrainImageGenerator(target, training_annotations_dict[target])
    training_image_generator.generate_training_images()

In [43]:
waldo_positive_images = []
waldo_negative_images = []

for positive_image in sorted(os.listdir(TrainImageGenerator.POSITIVE_TRAIN_IMG_DIR.format('waldo'))):
    img_path = os.path.join(TrainImageGenerator.POSITIVE_TRAIN_IMG_DIR.format('waldo'), positive_image)
    image = cv2.imread(img_path)
    waldo_positive_images.append(image)
    
for negative_image in sorted(os.listdir(TrainImageGenerator.NEGATIVE_TRAIN_IMG_DIR.format('waldo'))):
    img_path = os.path.join(TrainImageGenerator.NEGATIVE_TRAIN_IMG_DIR.format('waldo'), negative_image)
    image = cv2.imread(img_path)
    waldo_negative_images.append(image)

124 5800


In [44]:
hog_feature_params_rgb = {
    'orientations': 12,
    'transform_sqrt': True,
    'visualize': True
}

hog_rgb_feature_generator = FeatureGenerator('rgb', hog_feature_params_rgb)

waldo_positive_features = []
waldo_negative_features = []

for positive_img in waldo_positive_images:
    waldo_positive_features.append(hog_rgb_feature_generator.generate_features(positive_img))
    
for negative_img in waldo_negative_images:
    waldo_negative_features.append(hog_rgb_feature_generator.generate_features(negative_img))

In [66]:
unscaled_x = np.vstack((waldo_positive_features, waldo_negative_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack((np.ones(len(waldo_positive_features)), np.zeros(len(waldo_negative_features))))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = np.random.randint(1, 100))

-7.659080611469196e-17


In [67]:
svc = LinearSVC(max_iter=50000)
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)

In [76]:
for prob in sorted(svc.decision_function(x_test), reverse=True):
    print(prob)

1.3012802088081497
1.2340017470717615
1.1698934548549516
1.099359960169687
0.9912764755767056
0.9030836343163435
0.8754242560339764
0.8148240488335339
0.7058989086985439
0.699685777700032
0.4761237516353829
0.35759223651782457
0.29649254542653725
0.2935652628872618
0.10587881097057528
0.09500454651885448
0.058601337677659515
0.007360910609405691
0.006298366180939485
-0.02452154660032857
-0.08422542770130459
-0.11916861316951888
-0.1406696446871718
-0.16396023255174574
-0.18842668927273376
-0.20259273551013335
-0.2330742162816093
-0.24928780879269352
-0.2632349270403114
-0.2693324509039947
-0.2900134980902198
-0.29399959341890547
-0.29499822202914405
-0.3100428097159573
-0.3177334857316567
-0.31793801920607434
-0.33261229430888584
-0.33941369265213983
-0.35029623751620953
-0.36244951374505263
-0.36686061657143143
-0.36917588035317295
-0.380995177925752
-0.3819323309353705
-0.38545210792524354
-0.3919787734578539
-0.39602083909625097
-0.396976820882597
-0.40134870886549745
-0.40273862375